In [1]:
import torch
import numpy as np
import scipy
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from skimage import io, transform
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn import decomposition
from sklearn import svm
from matplotlib import cm
import torch.nn.functional as F
from torch import nn
import math
import pandas as pd
import os
import shapedimutils as s

%matplotlib inline

root = '/mnt/neurosphere/serenceslab2/maggie/shapeDim/'

image_dir = os.path.join(root,'Stimuli','AmpGrid3_adj_full_grey_small')
save_model_dir = os.path.join(root,'Modeling','saved_models')
if not os.path.exists(save_model_dir):
    os.makedirs(save_model_dir)
    
csv_file = os.path.join(image_dir, 'shape_labels_all.csv')
shape_labels = pd.read_csv(csv_file)

In [4]:
# testing out the dataset split generation

trn, val, tst  = s.get_dataset_splits(image_dir)
print([len(trn), len(val), len(tst)])
print(trn[0]['task_labels'])
print(val[0]['task_labels'])
print(tst[0]['task_labels'])
# for tt in range(10):
#     print(tst[tt]['task_labels'][0])
print(np.shape(trn[0]['image']))

[2107, 234, 260]
[1 1 0]
[0 0 0]
[0 0 0]
torch.Size([3, 224, 224])


In [2]:
# test the fine-tuning process
s.fine_tune_alexnet_binary(image_dir, save_model_dir, task=0, maxsteps=5)

step 0, trn loss = 1.09290, trn accuracy = 0.41000
step 1, trn loss = 1.00200, trn accuracy = 0.36000
step 2, trn loss = 0.97736, trn accuracy = 0.40000
step 3, trn loss = 0.80643, trn accuracy = 0.52000
step 4, trn loss = 0.87658, trn accuracy = 0.41000
step 4, val loss = 0.84007, val accuracy = 0.35
saving to /mnt/neurosphere/serenceslab2/maggie/shapeDim/Modeling/saved_models/AlexNet_finetune_task0.pt ...

saved


In [3]:
# test the evaluation function for the fine-tuned model
s.eval_alexnet_fine_tune(image_dir, save_model_dir, task=0)

loading from /mnt/neurosphere/serenceslab2/maggie/shapeDim/Modeling/saved_models/AlexNet_finetune_task0.pt

step 4, val loss = 0.84007, val accuracy = 0.35
step 4, test loss = 0.88342, test accuracy = 0.33


In [ ]:
## OLD
# what object category does it think the shapes are? 
final_activ = out.detach().numpy()
np.shape(final_activ)
np.argmax(final_activ,1)

array([530, 530, 530, 530, 530, 530, 530, 530, 530, 530, 530, 530, 818,
       530, 530, 530])

In [ ]:
## OLD
# try to classify category based on activs at different layers

layers2do = np.arange(0,13,1)
axis2discrim = 0  # 0 or 1
center = 2.5  # center of shape space is the "boundary"
image_labels = np.int64(coords2load[:,axis2discrim]>center) # create binary labels
image_inds = np.arange(0,nIms)

for ll in layers2do:
  # first, reshape to [nIms x nUnits]
  # disregarding difference between channels/spatial dims for now
  nUnitsTotal = np.prod(np.shape(activ[ll])[1:])
  activ_full = np.reshape(activ[ll].numpy(),[nIms, nUnitsTotal])
  np.shape(activ_full)

  # cross validated decoding leaving one im out at a time
  predlabs = np.zeros(np.shape(image_labels))
  for ii in range(nIms):
    
    trnlabs = image_labels[image_inds!=ii]
    tstlabs = image_labels[image_inds==ii]
    trndat = activ_full[image_inds!=ii,:]
    tstdat = activ_full[image_inds==ii,:]

    # train/test SVM classifier
    classifier = svm.SVC()
    classifier.fit(trndat, trnlabs)
    pred = classifier.predict(tstdat)
    predlabs[ii] = pred

  acc = np.mean(predlabs==image_labels)
  print('Layer %s: acc = %.2f'%(layer_names[ll],acc))
  

Layer 1_Conv2d: acc = 0.56
Layer 1_ReLU: acc = 0.56
Layer 1_MaxPool2d: acc = 0.94
Layer 2_Conv2d: acc = 0.94
Layer 2_ReLU: acc = 0.94
Layer 2_MaxPool2d: acc = 1.00
Layer 3_Conv2d: acc = 1.00
Layer 3_ReLU: acc = 1.00
Layer 4_Conv2d: acc = 1.00
Layer 4_ReLU: acc = 1.00
Layer 5_Conv2d: acc = 0.94
Layer 5_ReLU: acc = 0.94
Layer 5_MaxPool2d: acc = 0.88
